In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

In [10]:
# Iniciar el navegador (asegúrate de tener un controlador como ChromeDriver instalado)
driver = webdriver.Chrome()

# Navegar a la página
driver.get("https://www.baseball-reference.com/leagues/majors/2024-schedule.shtml")

# Esperar a que el elemento esté presente
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "section_wrapper")))

# Obtener el HTML de la página
page_content = driver.page_source

# Parsear el HTML con BeautifulSoup
soup = BeautifulSoup(page_content, "html.parser")

# Buscar la tabla de bateo
tabel_content = soup.find("div",{"class":"section_content"}).find_all("em")

driver.quit()

In [2]:
# Iniciar el navegador (asegúrate de tener un controlador como ChromeDriver instalado)
driver = webdriver.Chrome()

# Navegar a la página
driver.get("https://www.baseball-reference.com/leagues/majors/2024-schedule.shtml")

# Esperar a que el elemento esté presente
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "section_wrapper")))

# Obtener el HTML de la página
page_content = driver.page_source

# Parsear el HTML con BeautifulSoup
soup = BeautifulSoup(page_content, "html.parser")

# Buscar la tabla de bateo
table_content = soup.find("div",{"class":"section_content"})

#Buscar los boxscores

boxscores = table_content.find_all("em")


driver.quit()

In [3]:
# Extraer los href de los enlaces dentro de <em>
hrefs = [a.find("a")["href"] for a in boxscores if a.find("a")]

In [4]:
def match_scrapping(soup):

    #Get visiting and local teams
    vteam = soup.find("div",{"class":"scorebox"}).find_all("strong")[0].text.strip()
    lteam = soup.find("div",{"class":"scorebox"}).find_all("strong")[1].text.strip()
    
    vteam_id= vteam.replace(' ', '').replace('.', '')
    lteam_id= lteam.replace(' ', '').replace('.', '')

    #Get the metadata of the game
    date_content = soup.find("div",{"class":"scorebox_meta"}).find_all("div")

    date=date_content[0].text
    time=date_content[1].text
    attendance=date_content[2].text
    venue=date_content[3].text

    #Get the data for the visiting team
    vteam_content = soup.find(id=f"{vteam_id}batting")
    vteam_content_tr = vteam_content.find_all("tr")

    l = []
    for tr in vteam_content_tr:
        td = tr.find_all('td')
        th = tr.find_all('th')
        row = [tr.text for tr in td]
        team  = [tr.text for tr in th]
        complete_row = team + row
        l.append(complete_row)

    batting_vteam=pd.DataFrame(l[1:-1], columns=l[0])

    batting_vteam["Date"]=date
    batting_vteam["Time"]=time
    batting_vteam["Attendace"]=attendance
    batting_vteam["Venue"]=venue
    batting_vteam["Team"]=vteam

    #Get the data for the local team
    lteam_content = soup.find(id=f"{lteam_id}batting")
    lteam_content_tr = lteam_content.find_all("tr")

    l=[]
    for tr in lteam_content_tr:
        td = tr.find_all('td')
        th = tr.find_all('th')
        row = [tr.text for tr in td]
        team  = [tr.text for tr in th]
        complete_row = team + row
        l.append(complete_row)

    batting_lteam=pd.DataFrame(l[1:-1], columns=l[0])

    batting_lteam["Date"]=date
    batting_lteam["Time"]=time
    batting_lteam["Attendace"]=attendance
    batting_lteam["Venue"]=venue
    batting_lteam["Team"]=lteam

    batting_match_df=pd.concat([batting_vteam,batting_lteam], ignore_index=True)

    # Convert non-numeric values in the 'AB' column to NaN
    batting_match_df["AB"] = pd.to_numeric(batting_match_df["AB"], errors='coerce')

    # Drop rows with NaN in the 'AB' column
    batting_match_df = batting_match_df.dropna(subset=["AB"])


    #Get the ´pitching data for the visiting team
    vteam_content = soup.find(id=f"{vteam_id}pitching")
    vteam_content_tr = vteam_content.find_all("tr")

    l = []
    for tr in vteam_content_tr:
        td = tr.find_all('td')
        th = tr.find_all('th')
        row = [tr.text for tr in td]
        team  = [tr.text for tr in th]
        complete_row = team + row
        l.append(complete_row)

    pitching_vteam=pd.DataFrame(l[1:-1], columns=l[0])

    pitching_vteam["Date"]=date
    pitching_vteam["Time"]=time
    pitching_vteam["Attendace"]=attendance
    pitching_vteam["Venue"]=venue
    pitching_vteam["Team"]=vteam

    #Get the data for the local team
    lteam_content = soup.find(id=f"{lteam_id}pitching")
    lteam_content_tr = lteam_content.find_all("tr")

    l=[]
    for tr in lteam_content_tr:
        td = tr.find_all('td')
        th = tr.find_all('th')
        row = [tr.text for tr in td]
        team  = [tr.text for tr in th]
        complete_row = team + row
        l.append(complete_row)

    pitching_lteam=pd.DataFrame(l[1:-1], columns=l[0])

    pitching_lteam["Date"]=date
    pitching_lteam["Time"]=time
    pitching_lteam["Attendace"]=attendance
    pitching_lteam["Venue"]=venue
    pitching_lteam["Team"]=lteam


    pitching_match_df=pd.concat([pitching_vteam,pitching_lteam], ignore_index=True)
    
    return batting_match_df , pitching_match_df

In [6]:
pitching_stats_list=[]
batting_stats_list=[]

# Iniciar el navegador (asegúrate de tener un controlador como ChromeDriver instalado)
driver = webdriver.Chrome()

try:

    for match in hrefs:
        # Navegar a la página

        print(f"Procesando:{match}")

        driver.get(f"https://www.baseball-reference.com{match}")

        # Esperar a que el elemento esté presente
        WebDriverWait(driver, 10)#.until(EC.presence_of_element_located((By.CLASS_NAME, "section_wrapper")))

        # Obtener el HTML de la página
        page_content = driver.page_source

        # Parsear el HTML con BeautifulSoup
        soup = BeautifulSoup(page_content, "html.parser")

        df_batting , df_pitching=match_scrapping(soup)

        pitching_stats_list.append(df_pitching)
        batting_stats_list.append(df_batting)

        # Pausa entre solicitudes para evitar sobrecargar el servidor
        time.sleep(5)
finally:  
    driver.quit()

# Concatenar todos los DataFrames acumulados
batting_stats = pd.concat(batting_stats_list, ignore_index=True)
pitching_stats = pd.concat(pitching_stats_list, ignore_index=True)

Procesando:/boxes/SDN/SDN202403200.shtml
Procesando:/boxes/LAN/LAN202403210.shtml
Procesando:/boxes/ARI/ARI202403280.shtml
Procesando:/boxes/BAL/BAL202403280.shtml
Procesando:/boxes/CHA/CHA202403280.shtml
Procesando:/boxes/CIN/CIN202403280.shtml
Procesando:/boxes/HOU/HOU202403280.shtml
Procesando:/boxes/KCA/KCA202403280.shtml
Procesando:/boxes/LAN/LAN202403280.shtml
Procesando:/boxes/MIA/MIA202403280.shtml
Procesando:/boxes/OAK/OAK202403280.shtml
Procesando:/boxes/SDN/SDN202403280.shtml
Procesando:/boxes/SEA/SEA202403280.shtml
Procesando:/boxes/TBA/TBA202403280.shtml
Procesando:/boxes/TEX/TEX202403280.shtml
Procesando:/boxes/ARI/ARI202403290.shtml
Procesando:/boxes/HOU/HOU202403290.shtml
Procesando:/boxes/LAN/LAN202403290.shtml
Procesando:/boxes/MIA/MIA202403290.shtml
Procesando:/boxes/NYN/NYN202403290.shtml
Procesando:/boxes/OAK/OAK202403290.shtml
Procesando:/boxes/PHI/PHI202403290.shtml
Procesando:/boxes/SDN/SDN202403290.shtml
Procesando:/boxes/SEA/SEA202403290.shtml
Procesando:/boxe

Procesando:/boxes/SEA/SEA202404120.shtml
Procesando:/boxes/TBA/TBA202404120.shtml
Procesando:/boxes/TOR/TOR202404120.shtml
Procesando:/boxes/ARI/ARI202404130.shtml
Procesando:/boxes/BAL/BAL202404130.shtml
Procesando:/boxes/BOS/BOS202404130.shtml
Procesando:/boxes/CHA/CHA202404130.shtml
Procesando:/boxes/CLE/CLE202404131.shtml
Procesando:/boxes/CLE/CLE202404132.shtml
Procesando:/boxes/DET/DET202404131.shtml
Procesando:/boxes/DET/DET202404132.shtml
Procesando:/boxes/HOU/HOU202404130.shtml
Procesando:/boxes/LAN/LAN202404130.shtml
Procesando:/boxes/MIA/MIA202404130.shtml
Procesando:/boxes/NYN/NYN202404130.shtml
Procesando:/boxes/OAK/OAK202404130.shtml
Procesando:/boxes/PHI/PHI202404130.shtml
Procesando:/boxes/SEA/SEA202404130.shtml
Procesando:/boxes/TBA/TBA202404130.shtml
Procesando:/boxes/TOR/TOR202404130.shtml
Procesando:/boxes/ARI/ARI202404140.shtml
Procesando:/boxes/BAL/BAL202404140.shtml
Procesando:/boxes/BOS/BOS202404140.shtml
Procesando:/boxes/CHA/CHA202404140.shtml
Procesando:/boxe

Procesando:/boxes/COL/COL202404270.shtml
Procesando:/boxes/DET/DET202404270.shtml
Procesando:/boxes/MIA/MIA202404270.shtml
Procesando:/boxes/MIL/MIL202404270.shtml
Procesando:/boxes/NYN/NYN202404270.shtml
Procesando:/boxes/SDN/SDN202404270.shtml
Procesando:/boxes/SEA/SEA202404270.shtml
Procesando:/boxes/SFN/SFN202404270.shtml
Procesando:/boxes/TEX/TEX202404270.shtml
Procesando:/boxes/TOR/TOR202404270.shtml
Procesando:/boxes/ANA/ANA202404280.shtml
Procesando:/boxes/ATL/ATL202404280.shtml
Procesando:/boxes/BAL/BAL202404280.shtml
Procesando:/boxes/BOS/BOS202404280.shtml
Procesando:/boxes/CHA/CHA202404280.shtml
Procesando:/boxes/COL/COL202404280.shtml
Procesando:/boxes/DET/DET202404280.shtml
Procesando:/boxes/MIA/MIA202404280.shtml
Procesando:/boxes/MIL/MIL202404280.shtml
Procesando:/boxes/NYN/NYN202404280.shtml
Procesando:/boxes/SDN/SDN202404280.shtml
Procesando:/boxes/SEA/SEA202404280.shtml
Procesando:/boxes/SFN/SFN202404280.shtml
Procesando:/boxes/TEX/TEX202404280.shtml
Procesando:/boxe

Procesando:/boxes/DET/DET202405120.shtml
Procesando:/boxes/MIA/MIA202405120.shtml
Procesando:/boxes/MIL/MIL202405120.shtml
Procesando:/boxes/NYN/NYN202405120.shtml
Procesando:/boxes/PIT/PIT202405120.shtml
Procesando:/boxes/SDN/SDN202405120.shtml
Procesando:/boxes/SEA/SEA202405120.shtml
Procesando:/boxes/SFN/SFN202405120.shtml
Procesando:/boxes/TBA/TBA202405120.shtml
Procesando:/boxes/TOR/TOR202405120.shtml
Procesando:/boxes/ANA/ANA202405130.shtml
Procesando:/boxes/ARI/ARI202405130.shtml
Procesando:/boxes/ATL/ATL202405130.shtml
Procesando:/boxes/BAL/BAL202405130.shtml
Procesando:/boxes/BOS/BOS202405130.shtml
Procesando:/boxes/DET/DET202405130.shtml
Procesando:/boxes/HOU/HOU202405130.shtml
Procesando:/boxes/MIL/MIL202405130.shtml
Procesando:/boxes/NYN/NYN202405130.shtml
Procesando:/boxes/SDN/SDN202405130.shtml
Procesando:/boxes/SEA/SEA202405130.shtml
Procesando:/boxes/SFN/SFN202405130.shtml
Procesando:/boxes/TEX/TEX202405130.shtml
Procesando:/boxes/ANA/ANA202405140.shtml
Procesando:/boxe

Procesando:/boxes/CIN/CIN202405270.shtml
Procesando:/boxes/COL/COL202405270.shtml
Procesando:/boxes/MIL/MIL202405270.shtml
Procesando:/boxes/MIN/MIN202405270.shtml
Procesando:/boxes/SDN/SDN202405270.shtml
Procesando:/boxes/SEA/SEA202405270.shtml
Procesando:/boxes/SFN/SFN202405270.shtml
Procesando:/boxes/ANA/ANA202405280.shtml
Procesando:/boxes/ATL/ATL202405280.shtml
Procesando:/boxes/BAL/BAL202405280.shtml
Procesando:/boxes/CHA/CHA202405280.shtml
Procesando:/boxes/CIN/CIN202405280.shtml
Procesando:/boxes/COL/COL202405280.shtml
Procesando:/boxes/MIL/MIL202405280.shtml
Procesando:/boxes/MIN/MIN202405280.shtml
Procesando:/boxes/NYN/NYN202405281.shtml
Procesando:/boxes/NYN/NYN202405282.shtml
Procesando:/boxes/SDN/SDN202405280.shtml
Procesando:/boxes/SEA/SEA202405280.shtml
Procesando:/boxes/SFN/SFN202405280.shtml
Procesando:/boxes/TBA/TBA202405280.shtml
Procesando:/boxes/TEX/TEX202405280.shtml
Procesando:/boxes/ANA/ANA202405290.shtml
Procesando:/boxes/ATL/ATL202405290.shtml
Procesando:/boxe

Procesando:/boxes/SEA/SEA202406110.shtml
Procesando:/boxes/SFN/SFN202406110.shtml
Procesando:/boxes/SLN/SLN202406110.shtml
Procesando:/boxes/TBA/TBA202406110.shtml
Procesando:/boxes/ARI/ARI202406120.shtml
Procesando:/boxes/BAL/BAL202406120.shtml
Procesando:/boxes/BOS/BOS202406120.shtml
Procesando:/boxes/CIN/CIN202406120.shtml
Procesando:/boxes/DET/DET202406120.shtml
Procesando:/boxes/KCA/KCA202406120.shtml
Procesando:/boxes/LAN/LAN202406120.shtml
Procesando:/boxes/MIL/MIL202406120.shtml
Procesando:/boxes/MIN/MIN202406120.shtml
Procesando:/boxes/NYN/NYN202406120.shtml
Procesando:/boxes/SDN/SDN202406120.shtml
Procesando:/boxes/SEA/SEA202406120.shtml
Procesando:/boxes/SFN/SFN202406120.shtml
Procesando:/boxes/SLN/SLN202406120.shtml
Procesando:/boxes/TBA/TBA202406120.shtml
Procesando:/boxes/ARI/ARI202406130.shtml
Procesando:/boxes/BAL/BAL202406130.shtml
Procesando:/boxes/BOS/BOS202406130.shtml
Procesando:/boxes/DET/DET202406130.shtml
Procesando:/boxes/KCA/KCA202406130.shtml
Procesando:/boxe

Procesando:/boxes/KCA/KCA202406260.shtml
Procesando:/boxes/MIL/MIL202406260.shtml
Procesando:/boxes/NYN/NYN202406260.shtml
Procesando:/boxes/SDN/SDN202406260.shtml
Procesando:/boxes/SFN/SFN202406260.shtml
Procesando:/boxes/SLN/SLN202406261.shtml
Procesando:/boxes/SLN/SLN202406262.shtml
Procesando:/boxes/TBA/TBA202406260.shtml
Procesando:/boxes/ANA/ANA202406270.shtml
Procesando:/boxes/ARI/ARI202406270.shtml
Procesando:/boxes/BAL/BAL202406270.shtml
Procesando:/boxes/CHA/CHA202406270.shtml
Procesando:/boxes/KCA/KCA202406270.shtml
Procesando:/boxes/PHI/PHI202406270.shtml
Procesando:/boxes/SFN/SFN202406270.shtml
Procesando:/boxes/SLN/SLN202406270.shtml
Procesando:/boxes/TOR/TOR202406270.shtml
Procesando:/boxes/ANA/ANA202406280.shtml
Procesando:/boxes/ARI/ARI202406280.shtml
Procesando:/boxes/ATL/ATL202406280.shtml
Procesando:/boxes/BAL/BAL202406280.shtml
Procesando:/boxes/BOS/BOS202406280.shtml
Procesando:/boxes/CHA/CHA202406280.shtml
Procesando:/boxes/KCA/KCA202406280.shtml
Procesando:/boxe

Procesando:/boxes/NYN/NYN202407110.shtml
Procesando:/boxes/PHI/PHI202407110.shtml
Procesando:/boxes/SFN/SFN202407110.shtml
Procesando:/boxes/TBA/TBA202407110.shtml
Procesando:/boxes/ANA/ANA202407120.shtml
Procesando:/boxes/ARI/ARI202407120.shtml
Procesando:/boxes/BAL/BAL202407120.shtml
Procesando:/boxes/BOS/BOS202407120.shtml
Procesando:/boxes/CHA/CHA202407120.shtml
Procesando:/boxes/CIN/CIN202407120.shtml
Procesando:/boxes/DET/DET202407120.shtml
Procesando:/boxes/HOU/HOU202407120.shtml
Procesando:/boxes/MIL/MIL202407120.shtml
Procesando:/boxes/NYN/NYN202407120.shtml
Procesando:/boxes/PHI/PHI202407120.shtml
Procesando:/boxes/SDN/SDN202407120.shtml
Procesando:/boxes/SFN/SFN202407120.shtml
Procesando:/boxes/SLN/SLN202407120.shtml
Procesando:/boxes/TBA/TBA202407120.shtml
Procesando:/boxes/ANA/ANA202407130.shtml
Procesando:/boxes/ARI/ARI202407130.shtml
Procesando:/boxes/BAL/BAL202407130.shtml
Procesando:/boxes/BOS/BOS202407130.shtml
Procesando:/boxes/CHA/CHA202407130.shtml
Procesando:/boxe

Procesando:/boxes/PHI/PHI202407290.shtml
Procesando:/boxes/SLN/SLN202407290.shtml
Procesando:/boxes/ANA/ANA202407300.shtml
Procesando:/boxes/ARI/ARI202407300.shtml
Procesando:/boxes/BAL/BAL202407300.shtml
Procesando:/boxes/BOS/BOS202407300.shtml
Procesando:/boxes/CHA/CHA202407300.shtml
Procesando:/boxes/CIN/CIN202407300.shtml
Procesando:/boxes/DET/DET202407300.shtml
Procesando:/boxes/HOU/HOU202407300.shtml
Procesando:/boxes/MIL/MIL202407300.shtml
Procesando:/boxes/NYN/NYN202407300.shtml
Procesando:/boxes/PHI/PHI202407300.shtml
Procesando:/boxes/SDN/SDN202407300.shtml
Procesando:/boxes/SFN/SFN202407300.shtml
Procesando:/boxes/SLN/SLN202407300.shtml
Procesando:/boxes/TBA/TBA202407300.shtml
Procesando:/boxes/ANA/ANA202407310.shtml
Procesando:/boxes/ARI/ARI202407310.shtml
Procesando:/boxes/BAL/BAL202407310.shtml
Procesando:/boxes/BOS/BOS202407310.shtml
Procesando:/boxes/CHA/CHA202407310.shtml
Procesando:/boxes/CIN/CIN202407310.shtml
Procesando:/boxes/HOU/HOU202407310.shtml
Procesando:/boxe

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=131.0.6778.71)
Stacktrace:
	GetHandleVerifier [0x00007FF714BB6CB5+28821]
	(No symbol) [0x00007FF714B23840]
	(No symbol) [0x00007FF7149C578A]
	(No symbol) [0x00007FF7149B230C]
	(No symbol) [0x00007FF7149B21D0]
	(No symbol) [0x00007FF7149C8131]
	(No symbol) [0x00007FF714A5ED12]
	(No symbol) [0x00007FF714A3F0A3]
	(No symbol) [0x00007FF714A0A778]
	(No symbol) [0x00007FF714A0B8E1]
	GetHandleVerifier [0x00007FF714EEFCAD+3408013]
	GetHandleVerifier [0x00007FF714F0741F+3504127]
	GetHandleVerifier [0x00007FF714EFB5FD+3455453]
	GetHandleVerifier [0x00007FF714C7BDBB+835995]
	(No symbol) [0x00007FF714B2EB5F]
	(No symbol) [0x00007FF714B2A814]
	(No symbol) [0x00007FF714B2A9AD]
	(No symbol) [0x00007FF714B1A199]
	BaseThreadInitThunk [0x00007FFD69AC259D+29]
	RtlUserThreadStart [0x00007FFD6B82AF38+40]


In [7]:
# Concatenar todos los DataFrames acumulados
batting_stat = pd.concat(batting_stats_list, ignore_index=True)
pitching_stats = pd.concat(pitching_stats_list, ignore_index=True)

batting_stat.head()

,Batting,AB,R,H,RBI,BB,SO,PA,BA,OBP,...,acLI,RE24,PO,A,Details,Date,Time,Attendace,Venue,Team
0,Mookie Betts SS,4.0,0,2,1,1,0,5,.500,.600,...,1.29,1.4,1,4,,"Wednesday, March 20, 2024",Start Time: 7:05 p.m. Local,"Attendance: 15,952",Venue: Gocheok Sky Dome,Los Angeles Dodgers
1,Shohei Ohtani DH,5.0,0,2,1,0,0,5,.400,.400,...,1.20,0.3,0,0,SB,"Wednesday, March 20, 2024",Start Time: 7:05 p.m. Local,"Attendance: 15,952",Venue: Gocheok Sky Dome,Los Angeles Dodgers
2,Freddie Freeman 1B,2.0,0,0,0,2,1,5,.000,.600,...,1.09,-0.6,9,1,HBP,"Wednesday, March 20, 2024",Start Time: 7:05 p.m. Local,"Attendance: 15,952",Venue: Gocheok Sky Dome,Los Angeles Dodgers
3,Will Smith C,4.0,0,1,0,1,1,5,.250,.400,...,1.44,-0.2,6,0,,"Wednesday, March 20, 2024",Start Time: 7:05 p.m. Local,"Attendance: 15,952",Venue: Gocheok Sky Dome,Los Angeles Dodgers
4,Max Muncy 3B,3.0,1,1,0,2,2,5,.333,.600,...,1.76,0.3,0,1,,"Wednesday, March 20, 2024",Start Time: 7:05 p.m. Local,"Attendance: 15,952",Venue: Gocheok Sky Dome,Los Angeles Dodgers


In [9]:
pitching_stats.shape

(13925, 32)